In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,094 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,568 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:1

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-02 01:48:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.09MB/s    in 0.2s    

2022-12-02 01:48:14 (5.09 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
mobile_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True)
mobile_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
# Get the number of rows in the DataFrame.
mobile_df.count()

3514942

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = mobile_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = mobile_df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0715000322|Can-Am 715000322 ...|
|0984527281|Word Teasers Lear...|
|0991557557|PicoPen - Stainle...|
|1844257517|Yamaha FZ6 Fazer ...|
|1888000104|JLC Clarity Style...|
|1940825113|War Gaming - Stee...|
|3228963691|Zzzz... Sleeping ...|
|3456495633|Motorcycle First ...|
|3696387606|JLC 9005 CREE LED...|
|3696387622|JLC CREE 5W High ...|
|4678615123|SEM Paints (SEM15...|
|4678615173|Color Coat Camel ...|
|477961421X|Subaru Impreza No...|
|4891071478|Subaru Impreza GC...|
|4891075627|Toyota Mark II/Ch...|
|5888000272|Elite Mailers Nis...|
|5917025029|Nelson Rigg Weath...|
|5926025419|Joe Rocket RS-2 M...|
|5927026230|Nelson Rigg Unise...|
|592702632X|Nelson Rigg Unise...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = mobile_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    2583392|             4|
|     129005|             1|
|    3962209|             2|
|   16092109|             1|
|   46841294|             1|
|   42126527|             4|
|   47138302|             1|
|   10484128|             2|
|    5427648|             1|
|     580770|             1|
|   14286306|             6|
|   13880860|             1|
|   24095000|             2|
|   50189080|             1|
|   50583551|             4|
|   15562874|             1|
|   51068983|             3|
|   10245571|             6|
|   28932014|             8|
|   42891605|             4|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = mobile_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RAB23OVFNCXZQ|          1|            0|          0|   N|
|R3NORADVJO6IE6|          5|            0|          0|   N|
|R299F4SO98S5OO|          5|            1|          1|   N|
|R2DA9DOT03UW6I|          5|            2|          3|   N|
|R2OGCH681EQHU6|          5|            0|          0|   N|
| R2JMKIC16MHD7|          5|            2|          2|   N|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|
|R1N8XWFDK4QACP|          5|            0|          0|   N|
|R19VFLX6MRU0CX|          5|            0|          0|   N|
| RLTD8YDGTP9PT|          1|            0|          0|   N|
| R78B5LCFJ52SR|          4|            0|          0|   N|
|R33SKWL0HEQIQ9|          5|            0|          0|   N|
|R1HIOV0UX7P13D|          5|            0|          0|   N|
| R4WS0E0MSP9DH|          5|            

# Load

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ceq2jbmu0ic4.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root1", "password": "groot112299", "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='auto_review_id_shoes_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='auto_products', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='auto_customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='auto_vine_table', mode=mode, properties=config)